# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
 

from sklearn.metrics import classification_report, accuracy_score

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#cleaned data without non-attack values
X = pd.read_csv(root_path+'Data/Features.csv', index_col=0)
Y = pd.read_csv(root_path+'Data/Target.csv', index_col=0)

# Feature Engineering

In [0]:
# %%capture warnings1
# cluster and score
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.cluster import v_measure_score as v

score = []
for i in range(len(X.columns)): # loop number of features
  K = MiniBatchKMeans(n_clusters=len(Y[Y.columns[0]].unique()), random_state=0)
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = v(Y[Y.columns[0]].values,pred)
  score.append(s)
  


In [5]:
# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 


print("Index of Feature , adjusted_rand_score\n")
for i in range(len(X.columns)):
    print(li[i])


Index of Feature , adjusted_rand_score

(9, 0.5650863981587048)
(3, 0.52849101176294)
(18, 0.3392549877660737)
(41, 0.30179788787084577)
(40, 0.295412835133644)
(37, 0.2949329018075072)
(38, 0.2940409843619152)
(36, 0.2910448933160603)
(1, 0.2758371802253723)
(42, 0.27566801642539634)
(19, 0.2725704434126493)
(6, 0.26965238025644755)
(13, 0.26862274351835663)
(32, 0.2682361292134235)
(11, 0.26730576460575994)
(14, 0.2668434700603014)
(15, 0.2668296304474186)
(12, 0.2655601995652946)
(7, 0.2511535966575439)
(22, 0.2467951655072783)
(23, 0.24362679700297124)
(10, 0.23757560275438708)
(27, 0.2361957405160201)
(39, 0.23329808768902935)
(2, 0.22754251567950337)
(28, 0.2254912760173461)
(30, 0.22043090251573835)
(29, 0.20740855238776648)
(16, 0.20378397654292862)
(17, 0.2021035305818563)
(26, 0.20075624388919194)
(0, 0.16904586171222086)
(8, 0.16783845468354455)
(5, 0.15945140696556998)
(4, 0.1279175838128704)
(33, 0.07781006442732108)
(20, 0.07780729553793987)
(21, 0.0521466555567166)
(24, 

In [0]:
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]




In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

In [8]:
# RF w/out recursive Feature Elimination
x_train, x_test, y_train, y_test = train_test_split(X1, Y, random_state=11)
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred, digits=4))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                precision    recall  f1-score   support

      Analysis     0.6358    0.1664    0.2638       619
      Backdoor     0.6104    0.0829    0.1460       567
           DoS     0.3156    0.2699    0.2910      4075
      Exploits     0.6352    0.8075    0.7111     11236
       Fuzzers     0.8988    0.8655    0.8818      6127
       Generic     0.9972    0.9860    0.9915     53886
Reconnaissance     0.9213    0.7507    0.8273      3385
     Shellcode     0.6557    0.6186    0.6366       388
         Worms     1.0000    0.2105    0.3478        38

      accuracy                         0.8907     80321
     macro avg     0.7411    0.5287    0.5663     80321
  weighted avg     0.8941    0.8907    0.8879     80321



In [9]:
# %%capture warnings
# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(random_state=0, n_jobs=-1)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff

In [10]:
index = accuracy.index(max(accuracy))
# for i in accuracy:
#   print(i)
  

# print(accuracy)

print(max(accuracy), "max accuracy")

print(index, "index")


X3 = X2.iloc[:,0:len(X.columns)-index]

0.8989059557713557 max accuracy
24 index


# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X3, Y, test_size=0.2, random_state=11)

In [12]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [13]:
x = list(zip(X3.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X3.columns)):
    print(x[i])

('sbytes', 0.32595939391862944)
('ct_dst_src_ltm', 0.10798459568200748)
('service', 0.10584303792906954)
('smeansz', 0.09093058613140301)
('ct_src_dport_ltm', 0.05666178463992033)
('ct_dst_ltm', 0.05652450825651695)
('ct_dst_sport_ltm', 0.05300291699559518)
('dmeansz', 0.041543927891548864)
('Dload', 0.03856247597094035)
('ct_srv_dst', 0.030324626310188434)
('ct_srv_src', 0.02683909479290567)
('Dpkts', 0.01971939953090306)
('Spkts', 0.01625950137176931)
('sloss', 0.014529900440767237)
('dttl', 0.004829415727928675)
('dwin', 0.0034322739395898573)
('swin', 0.0030908277966941143)
('state', 0.0027831400488851654)
('ct_state_ttl', 0.0011785926247371418)


In [14]:
clf.score(x_train, y_train) 

0.9254355590485009

In [15]:
print("Accuracy")
clf.score(x_test, y_test) 

Accuracy


0.8989059557713557

In [16]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

                precision    recall  f1-score   support

      Analysis     0.7679    0.1737    0.2834       495
      Backdoor     0.7692    0.0853    0.1536       469
           DoS     0.3929    0.1740    0.2412      3235
      Exploits     0.6236    0.8836    0.7311      8944
       Fuzzers     0.9140    0.8661    0.8894      4846
       Generic     0.9967    0.9879    0.9922     43219
Reconnaissance     0.9210    0.7526    0.8283      2696
     Shellcode     0.7112    0.7290    0.7200       321
         Worms     0.5769    0.4688    0.5172        32

      accuracy                         0.8989     64257
     macro avg     0.7415    0.5690    0.5952     64257
  weighted avg     0.8999    0.8989    0.8903     64257

